### Visualize how factor is applied

In [ ]:
import pandas as pd
import numpy as np
import os
from io import StringIO
from functools import reduce

import matplotlib.pyplot as plt

INPUT_DIR = "/home/tomasmizera/school/diploma/src/data/reviews/only-long"

files_order = list(filter(lambda x: x, (map(lambda x: x.name if x.is_file() else '', os.scandir(INPUT_DIR)))))
files_order

In [ ]:
LOGS_DIR = "/home/tomasmizera/school/diploma/src/data/logs/experiment/v3-fix/"

In [ ]:
def load_files(path_to_files):
    
    def _read_text_file(filepath):
        with open(filepath, 'r') as f:
            lines = f.readlines()
            lines[0] = lines[0][2:] # remove leading hashtag
            lines[0] = lines[0].replace(',', ' ')
            return reduce(lambda a, b: a + b, lines)
    
    files_it = os.scandir(path_to_files)
    files_contents = {}
    
    for file in files_it:
        if file.is_file() and file.name.startswith('fp-'):
            factor = file.name.replace('fp-', '')
            factor = factor.replace('.csv', '')
            files_contents[factor] = _read_text_file(file.path)
        
    return dict(sorted(files_contents.items()))

In [ ]:
file_contents = load_files(LOGS_DIR)

In [ ]:
# 1. build dataframe from each

dfs = {}

for item in file_contents.items():
    factor = item[0]
    content = StringIO(item[1])
    df = pd.read_csv(content, delimiter=' ')
    dfs[factor] = df

# 2. filter silly examples

silly_files_mask = [
    files_order.index("review-low.txt-test-setup"),
    files_order.index("review-med.txt-test-setup"),
    files_order.index("review-top.txt-test-setup"),
    files_order.index("review-low.txt-test"),
    files_order.index("review-med.txt-test"),
    files_order.index("review-top.txt-test") 
]
silly_files_mask

In [ ]:
def process(df):
#     df = df.drop(silly_files_mask, errors='ignore')
    df = df.reset_index(drop=True)
    
    df['difSimple'] = df.originalP - df.simpleSP
    df['difCustom'] = df.originalP - df.customSP
    return df.difCustom.mean(), df.difSimple.mean()

def process_arr(dfs):
    data = {}
    for d in dfs.items():
        ix = d[0]
        data[int(ix)] = process(d[1])
    return data


In [ ]:
processed = process_arr(dfs)

In [ ]:
pdf = pd.DataFrame(processed).T
pdf = pdf.rename({0:'custom', 1:'textrank'}, axis=1)
pdf = pdf.sort_index()
pdf

In [ ]:
ax = pdf.plot(
    figsize=(15,8), 
    xlabel="Factor", 
    ylabel="Difference from original prediction", 
    fontsize=10, 
    title="Difference from original prediction using various factors (the smaller the best)"
);

ax.axhline(y=0, color='r', linestyle='--', label='vline1.5custom');

plt.show()

In [ ]:
pdf.T.plot.box(
    figsize=(15,8), 
    xlabel="Factor", 
    ylabel="Difference from original prediction", 
    fontsize=10, 
    title="Difference from original prediction using various factors (the smaller the best)"
);

In [ ]:
pdf.plot.box()

In [ ]:
pdf.describe()

**OUTCOME: For our method I choose factor of size 5!**